## Library used and deeclare data needed

In [79]:
import numpy as np
import csv
from numpy.typing import NDArray
from scipy.stats import zscore
from pytictoc import TicToc
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import accuracy_score

# from matilda.data.model import AlgorithmSummary
# from matilda.data.option import Opts

In [82]:
kernel_fcn = 'rbf'
opts_csv_fold = 5
with open('./data/algolabels.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            algo_labels = row

# prepare for y, transpose it
# y = pd.read_csv('./data/ybin.csv')
# y = y.values.tolist()

y = np.loadtxt('./data/ybin.csv', delimiter=',', skiprows=1)

# prepare for z, normalise it
z = pd.read_csv('./data/z.csv')
z_norm = zscore(z, axis = 0, ddof = 1)

ninst, nalgos = y.shape
w = np.ones((ninst, nalgos))

cvcmat = np.zeros((nalgos, 4))

### Training

In [91]:
def fit_libsvm(z, y, kkv, kernel_given):
    accuracy= dict()
    for k, v in kkv.items():
        train_index, test_index = v[0], v[1]
        # prepare training data
        x_train = [z[i] for i in train_index]
        y_train = [y[i] for i in train_index]
        # prepare test data
        x_test = [z[i] for i in test_index]
        y_test = [y[i] for i in test_index]
        svm = SVC(kernel=kernel_given, C=1.0, random_state = 0)
        svm.fit(x_train, y_train)
        y_pred = svm.predict(x_test)
        # calculate accuracy
        accuracy[k] = accuracy_score(y_test, y_pred)
        
    return accuracy

In [47]:
def fit_matsvm(z, y, w, skf, kernel_given, params):
    # Set up parallel workers in pool
    
    # Check if hyperparameter is given by user
    if(np.isnan(params)):
        # Initialize a random number generator
        np.random.seed(0)

        # Scikit-learn lib need to ensuring data contiguity
        z = np.ascontiguousarray(z)
        y = np.ascontiguousarray(y)
        w = np.ascontiguousarray(w)

        # Debug shape and data type
        # print(w.flags)

        # Retrieve default hyperparameters for fitcsvm and sets the range for the box constraint (C) and kernel scale
        # Define the range for C and gamma in a logarithmic scale
        param_grid = {
        # Generates 15 numbers between 2^-10 and 2^4
        'C': np.logspace(-10, 4, base=2, num=15),
        'gamma': np.logspace(-10, 4, base=2, num=15)  
        }

        # cache_size: maximal, ?????class_weight='balanced'
        svm_model = SVC(kernel=kernel_given, cache_size=2000, class_weight='balanced', probability=True)

        # Used for exhaustive search over specified parameter values for the SVM. The param_grid defines 
        # the range over which C and gamma will be tuned.
        # GridSearchCV for optimizing the hyperparameters
        grid_search = GridSearchCV(
            estimator=svm_model, 
            param_grid=param_grid, 
            # 'roc_auc' measures the area under the receiver operating characteristic curve, which is a 
            # good choice for binary classification problems, especially with imbalanced classes.
            scoring='roc_auc', 
            cv=skf, 
            verbose=0
            #, n_jobs=nworkers if nworkers != 0 else None,
            )

        # Fit GridSearchCV
        grid_search.fit(z, y, sample_weight=w)

        # Retrieve the best model and hyperparameters
        best_svm = grid_search.best_estimator_
        best_C = grid_search.best_params_['C']
        best_g = grid_search.best_params_['gamma']

        # Calibrate the probability model
        calibrated_svm = CalibratedClassifierCV(best_svm, cv='prefit')
        calibrated_svm.fit(z, y, sample_weight=w)
        
        # Making predictions on the training data
        y_sub = calibrated_svm.predict(z)
        p_sub = calibrated_svm.predict_proba(z)[:, 1]

        t_y_sub = best_svm.predict(z)
        t_p_sub = best_svm.predict_proba(z)[:, 1]

        # Making predictions on the same data to simulate resubstitution prediction
        y_hat = y_sub
        p_hat = p_sub
        
        print("svm: ", calibrated_svm)
        print("Best C:", best_C)
        print("Best gamma:", best_g)


    return calibrated_svm, y_sub #, p_sub, y_hat, p_hat, best_C, best_g

### MATSVM


In [86]:
t = TicToc()
t.tic()

for i in range(nalgos):
    t_inner = TicToc()
    t_inner.tic()

    state = np.random.get_state()
    np.random.seed(0)  # equivalent to MATLAB's rng('default') ?

    # REQUIRE: Test case for validation the result
    # y_b = [row[i] for row in y]
    y_b = y[:, i]
    skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)
    
    # kkv= dict()
    # for i, (train_index, test_index) in enumerate(skf.split(np.zeros(len(y_b)), y_b)):
    #     kkv[i] = [train_index.tolist(), test_index.tolist()]
    # start training using svm
    # svm_res = fit_libsvm(z_norm, y_b, kkv, kernel_fcn)
    
    svm_res, y_sub = fit_matsvm(z_norm, y_b, w[:, i], skf, kernel_fcn, np.nan)
    aux = confusion_matrix(y_b, y_sub)
    print("------------aux-----------")
    print(aux)
    # 66    17   16   113

    # np.prod(aux.shape) != 4 is False
    cvcmat[i, :] = aux.flatten()
    models_left = nalgos - (i + 1)
    print(f"    -> PYTHIA has trained a model for {algo_labels[i]}, there are {models_left} models left to train.")

    print(f"      -> Elapsed time: {t_inner.tocvalue():.2f}s")

    

svm:  CalibratedClassifierCV(cv='prefit',
                       estimator=SVC(C=8.0, cache_size=2000,
                                     class_weight='balanced', gamma=0.0078125,
                                     probability=True))
Best C: 8.0
Best gamma: 0.0078125
------------aux-----------
[[ 65  17]
 [ 16 113]]
    -> PYTHIA has trained a model for NB, there are 9 models left to train.
      -> Elapsed time: 16.15s
svm:  CalibratedClassifierCV(cv='prefit',
                       estimator=SVC(C=16.0, cache_size=2000,
                                     class_weight='balanced', gamma=0.5,
                                     probability=True))
Best C: 16.0
Best gamma: 0.5
------------aux-----------
[[ 63  20]
 [ 10 118]]
    -> PYTHIA has trained a model for LDA, there are 8 models left to train.
      -> Elapsed time: 16.77s
svm:  CalibratedClassifierCV(cv='prefit',
                       estimator=SVC(C=8.0, cache_size=2000,
                                     class_weight=

### Estimation

In [88]:
  
tn, fp, fn, tp = cvcmat[:, 0], cvcmat[:, 1], cvcmat[:, 2], cvcmat[:, 3]
print(tn.dtype)  
print(tn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tp + tn) / ninst

print(precision)
print(recall)
print(accuracy)

float64
[ 65.  63. 111.  54.  48.  51.  54.  85.  54.  48.]
[0.86923077 0.85507246 0.67948718 0.95973154 0.94155844 0.96078431
 0.96       0.875      0.95333333 0.8807947 ]
[0.87596899 0.921875   0.70666667 0.94701987 0.94155844 0.95454545
 0.95364238 0.875      0.95333333 0.91724138]
[0.8436019  0.85781991 0.77725118 0.93364929 0.91469194 0.93838863
 0.93838863 0.86729858 0.93364929 0.85781991]


### LIBSVM

In [92]:
y = pd.read_csv('./data/ybin.csv')
y = y.values.tolist()

z = pd.read_csv('./data/z.csv').values.tolist()
z_norm = zscore(z, axis = 0, ddof = 1)

for i in range(nalgos):
    t_inner = TicToc()
    t_inner.tic()

    state = np.random.get_state()
    np.random.seed(0)  # equivalent to MATLAB's rng('default') ?

    # REQUIRE: Test case for validation the result
    y_b = [row[i] for row in y]
    # y_b = y[:, i]
    skf = StratifiedKFold(n_splits = opts_csv_fold, shuffle = True, random_state = 0)
    
    kkv= dict()
    for i, (train_index, test_index) in enumerate(skf.split(np.zeros(len(y_b)), y_b)):
        kkv[i] = [train_index.tolist(), test_index.tolist()]
    
    # start training using svm
    svm_res = fit_libsvm(z_norm, y_b, kkv, kernel_fcn)

    # visualise accuracy score
for k, v in svm_res.items():
    print(f'{k} fold: accuracy score = {v}')

0 fold: accuracy score = 0.8837209302325582
1 fold: accuracy score = 0.9047619047619048
2 fold: accuracy score = 0.8333333333333334
3 fold: accuracy score = 0.8333333333333334
4 fold: accuracy score = 0.8571428571428571
